## *****For Demonstration purpose only, Please customize as per your enterprise security needs and compliances***** 
Disclaimer: By accessing this code, you acknowledge the code is made available for presentation and demonstration purposes only and that the code: (1) is not subject to SOC 1 and SOC 2 compliance audits; (2) is not designed or intended to be a substitute for the professional advice, diagnosis, treatment, or judgment of a certified financial services professional; (3) is not designed, intended or made available as a medical device; and (4) is not designed or intended to be a substitute for professional medical advice, diagnosis, treatment or judgement. Do not use this code to replace, substitute, or provide professional financial advice or judgment, or to replace, substitute or provide medical advice, diagnosis, treatment or judgement. You are solely responsible for ensuring the regulatory, legal, and/or contractual compliance of any use of the code, including obtaining any authorizations or consents, and any solution you choose to build that incorporates this code in whole or in part. 


In [1]:
#Import the required libraries
import pandas as pd
import yfinance as yf
#from yahoofinancials import YahooFinancials
from datetime import datetime

StatementMeta(FinanceSpk, 7, 1, Finished, Available)

In [2]:
# Read out CSV file with list of all tickers
nqSdf = spark.read.load('abfss://risk@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Portfolio/PortfolioCik.csv', 
    format='csv', 
    sep=",",
    header=True)

portfolioData = nqSdf.toPandas()
display(portfolioData)

StatementMeta(FinanceSpk, 7, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 09e61815-27f5-40c6-90b0-9a6073bfcd16)


/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  'JavaPackage' object is not callable
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.

In [3]:
from azure.storage.blob import BlobServiceClient, ContainerClient
from azure.storage.blob import BlobClient

accountName = "#STORAGE_ACCOUNT_NAME#"
accountKey = "#STORAGE_ACCOUNT_KEY#"
containerName = "risk"
connectionString = "DefaultEndpointsProtocol=https;AccountName=#STORAGE_ACCOUNT_NAME#;AccountKey=#STORAGE_ACCOUNT_KEY#;EndpointSuffix=core.windows.net"

StatementMeta(FinanceSpk, 7, 3, Finished, Available)

In [4]:
# Get the stock price for all tickers and write it to CSV file
#yahoo_financials = YahooFinancials(tickerTop10)

#data = yahoo_financials.get_historical_price_data(start_date='2019-01-01', 
#                                                  end_date='2019-01-31', 
#                                                  time_interval='daily')
finalColNames = ["Portfolio", "Sector", "Cik", "Date", "Ticker", "AdjClose"]

for index, ticker in portfolioData.iterrows():
    finalDf = pd.DataFrame(columns = finalColNames)
    stockData = yf.download(ticker['Ticker'], period="max", interval='1d', group_by='ticker', threads=True, progress=True)
    for index, row in stockData.iterrows():
        finalDf = finalDf.append({'Portfolio' : str(ticker['Portfolio']), 'Sector' : ticker['Sector'], 'Cik' : ticker['CIK'], 'Date' : index.date() , 'Ticker' : ticker['Ticker'], 'AdjClose' : row['Adj Close'] } , ignore_index=True)
    output = finalDf.to_csv (index=False, header=True, encoding = "utf-8")
    blobName = "PortfolioQuotes/HistoricalData/" + ticker['Ticker'] + ".csv"
    blob = BlobClient.from_connection_string(conn_str=connectionString, container_name=containerName, blob_name=blobName)
    blob.upload_blob(output)
    #blobService.create_blob_from_text(containerName, blobName, output)



StatementMeta(FinanceSpk, 7, 4, Finished, Available)

ClientAuthenticationError: Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:f4db2c04-001e-008a-1ed3-568195000000
Time:2021-06-01T10:47:42.7857588Z
ErrorCode:AuthenticationFailed
Error:None
AuthenticationErrorDetail:The MAC signature found in the HTTP request '4Pnl7E1FI6Uh62JyaePVF9WGh0BghyWccXR0buLGOUU=' is not the same as any computed signature. Server used following string to sign: 'PUT


341110

application/octet-stream



*


x-ms-blob-type:BlockBlob
x-ms-client-request-id:cad642dc-c2c6-11eb-9419-000d3afcab19
x-ms-date:Tue, 01 Jun 2021 10:47:42 GMT
x-ms-version:2019-12-12
/stfsiprod/risk/PortfolioQuotes/HistoricalData/AMZN.csv'.